# pyCCA

#### 191218

This notebook helps with doing Connected Concept Analysis in Python. It does (roughly) the same job as [Textometrica](http://textometrica.humlab.umu.se).

For a description of CCA, refer to this paper:

>Lindgren, S. (2016). ["Introducing Connected Concept Analysis: A network approach to big text datasets"](https://doi.org/10.1515/text-2016-0016). _Text & Talk: An Interdisciplinary Journal of Language, Discourse & Communication Studies_ 36(3): 341–362.

CCA is a workflow for combining manual thematic coding with a form of [NTA](http://www.casos.cs.cmu.edu/publications/protected/1995-1999/1995-1997/carley_1997_networktext.PDF).

#### 1. Setup

Import the required libraries.

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

Prepare for stopword removal.

Set as `keep_words` a list of standard stopwords, but that should not be removed.

Set as `extra_stops` a list of non standard stopwords that should be removed.

Stopwords will be loaded from nltk, removing the `keep_words` from the stoplist, and adding the `extra_stops` to it.

In [ ]:
# Set up stopwords
keep_words = ["of"] # name standard stopwords that should not be removed
stops = [i for i in stopwords.words('english') if not i in keep_words] # load standard stopwords
extra_stops = ["via", "thats", "i’ve", "dont", "cant","rt", "com", "twitter", "http", "https",\
               "pic", "www","de","fb","en","ly","html", "status", "us", "es", "st", "tt", "la"\
              "li","se", "bit", "ly"] # name non-standard stopwords to be removed
for i in extra_stops:
    stops.append(i)
stops = frozenset(stops)

Import the corpus from a file with one document per line.

In [ ]:
corpus = [doc.strip() for doc in open("docs.txt")]

#### 2. Vectorize and count

Vectorize the corpus, while removing stopwords, and only keeping words with >2 letters, and no numerical or special characters.

We use `ngram_range = (1,3)` to get unigrams, bigrams, and trigrams.

In [ ]:
%%time
# Vectorize and count
cv = CountVectorizer(ngram_range=(1, 3),
                     strip_accents = 'unicode',
                     stop_words=stops,
                     token_pattern="[a-zA-Z][a-zA-Z]+") # at least two letters, and no numerical or special characters
dtm = cv.fit_transform(corpus)

Get an ordered list of all token names.

In [ ]:
wordlist = cv.get_feature_names()

Get document frequencies for tokens (i.e. how many documents they occur in, no matter how many times).

In [ ]:
docfreqs = list(np.squeeze(np.asarray((dtm != 0).sum(0)))) # count number of non-zero document occurrences for each row (i.e. each word)

Save token idnumbers, token names, and document frequencies in a dataframe.

In [ ]:
countsDF = pd.DataFrame(zip(wordlist,docfreqs)).reset_index()
countsDF.columns = ["id","token", "DF"]

Keep only the top 3000 tokens, by document frequency.

In [ ]:
countsDF = countsDF.sort_values(by="DF", ascending=False).head(3000)

Present the user with a list of top tokens (document frequency) in the corpus.

In [ ]:
with open("words.txt", "w") as f:
    for term in countsDF.token:
        f.write(term + "\n")

#### 3. Word selection

<hr>
**Before continuing:**

- Open the `words.txt` file in an editor and delete all lines with words that you do _not_ want to keep for analysis. 
- Save the file with the same name, in the same directory.

<hr>

#### 4. Define concepts

In [ ]:
analysis_words = [w.strip() for w in open("words.txt", "r").readlines()]
df = pd.DataFrame(list(zip(analysis_words, analysis_words)), columns=['word','concept'])
df.to_csv("concepts.csv", index=None)

<hr>
**Before continuing:**

- Open the `concepts.csv` file in an editor and enter concept names in the second column. 
- Leave the header row (`word,concept`) as it is.
- If the word should not belong to a conceptual category, leave it as it is. 
- If you want to exclude the word from analysis, delete its entire row.
- Save the file with the same name in the same directory.

<hr>

In [ ]:
tokensDF = pd.DataFrame([w.strip() for w in open("words.txt", "r").readlines()])
tokensDF.columns = ['token']
finalDF = pd.merge(tokensDF,countsDF)

In [ ]:
tokenids_we_want = list(finalDF.id)

#### 5. Get co-occurrences

Get a smaller document-term matrix with only the tokens we want.

In [ ]:
dtm2 = dtm[:, tokenids_we_want]
dtm2.shape

Get co-occurrences, and write as `networkx` graph.

In [ ]:
cooc_matrix = (dtm2.T * dtm2) # this is cooccurrence matrix in sparse csr format
cooc_matrix.setdiag(0) # fill same word cooccurence to 0
cooc_matrix = cooc_matrix.todense() # convert sparse to dense

#### 6. Graph preparation

In [ ]:
G=nx.from_numpy_matrix(cooc_matrix)
print(nx.info(G))

<hr>
Keep only edges with a weight > than the `cutoff`.

The cell below can be iterated with different cutoffs to see the size of the resulting `G2` graph.

In [ ]:
cutoff = 1
top = [edge for edge in G.edges_iter(data=True) 
       if edge[2]['weight'] > cutoff]
G2 = nx.Graph(top)
print(nx.info(G2))

<hr>
Replace the numeric token labels with full text versions.

In [ ]:
labels = finalDF.token # an iterable of labels (in the right order)
H = nx.relabel_nodes(G2, labels)

In [ ]:
# Read the concepts.csv into a dictionary
import csv

reader = csv.reader(open("concepts.csv"))
next(reader, None) # skip the header line in the file
newlabels = {}
for row in reader:
    key = row[0]
    value = row[1]
    newlabels.update( {key : value} )

In [ ]:
I = nx.relabel_nodes(H, newlabels)

Save the graphs in Gephi format.

In [ ]:
nx.write_gexf(H, "pyCCA_words.gexf")
nx.write_gexf(I, "pyCCA_concepts.gexf")